In [1]:
#pip install keras
#pip install tensorflow
#https://towardsdatascience.com/a-single-function-to-streamline-image-classification-with-keras-bd04f5cfe6df
#https://www.youtube.com/watch?v=LhEMXbjGV_4.  ::: Image preparation for CNN Image Classifier with Keras
#https://www.youtube.com/watch?v=Zrt76AIbeh4   ::: Fine-Tuning MobileNet on Custom Data Set with TensorFlow's Keras API
#https://www.youtube.com/watch?v=4Tcqw5oIfIg   ::: image classifier using transfer learning 
# https://www.youtube.com/watch?v=8Nl-of5C3uA  ::: to save the model
#%pip install splitfolders
#import splitfolders

In [51]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.metrics import confusion_matrix 
%matplotlib inline

In [27]:
%pwd

'/Users/anks/Downloads/YOLLOO'

In [5]:
#%pwd
suv_img = len(os.listdir("car/suv"))
sedan_img = len(os.listdir("car/sedan"))
print(suv_img)
print(sedan_img)

771
795


In [6]:

suv_data = os.listdir("car/suv")
sedan_data = os.listdir("car/sedan")

from sklearn.model_selection import train_test_split
train_suv, test_suv = train_test_split(suv_data, test_size=0.2, random_state=1)
train_sedan, test_sedan = train_test_split(sedan_data, test_size=0.2, random_state=1)


In [7]:
######      COPY IMAGE INTO FROM  FOLDER "SUV" TO "TRAIN/SUV" & "TEST/SUV"     #####
        
##############      TRIINING DATA CREATION      ######################


#src_dir = "car/suv/"

#dst_dir_suv = "car/train/suv"
#for i in train_suv:
    #shutil.copy(src_dir+i, dst_dir_suv)
    

#dst_dir_suv1 = "car/test/suv"
#for i in test_suv:
    #shutil.copy(src_dir+i, dst_dir_suv1) 


In [11]:
######      COPY IMAGE INTO FROM  FOLDER "SEDAN" TO "TRAIN/SEDAN" & "TEST/SEDAN"     #####

#src_dir = "car/sedan/"

#dst_dir_sedan = "car/train/sedan"
#for i in train_sedan:
    #shutil.copy(src_dir+i, dst_dir_sedan)
    

#dst_dir_sedan1 = "car/test/sedan"
#for i in test_sedan:
    #shutil.copy(src_dir+i, dst_dir_sedan1) 


In [34]:
test_suv, val_suv = train_test_split(test_suv, test_size=0.1, random_state=1)
test_sedan, val_sedan = train_test_split(test_sedan, test_size=0.1, random_state=1)

In [36]:
#####

#src_dir = "car/sedan/"


#dst_dir = "car/test/sedan"
#for i in test_sedan:
    #shutil.copy(src_dir+i, dst_dir)

    
#dst_dir = "car/val/sedan"   
#for i in val_sedan:
    #shutil.copy(src_dir+i, dst_dir) 

In [37]:
#%pwd
print(len(os.listdir("car/train/suv")))
print(len(os.listdir("car/train/sedan")))
print(len(os.listdir("car/test/suv")))
print(len(os.listdir("car/test/sedan")))
print(len(os.listdir("car/val/suv")))
print(len(os.listdir("car/val/sedan")))


616
636
139
143
16
16


In [38]:
train_path = "car/train"
test_path = "car/test"
val_path = "car/val"

In [39]:
#ImageDataGenerator generates batches of tensor image data = this is formate which is required to read by keras model
train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size =(224,224),classes =['suv','sedan'], batch_size =10)
valid_batches = ImageDataGenerator().flow_from_directory(val_path,target_size =(224,224),classes =['suv','sedan'], batch_size =10)
test_batches = ImageDataGenerator().flow_from_directory(test_path,target_size =(224,224),classes =['suv','sedan'], batch_size =10, shuffle = False)


Found 1252 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Found 282 images belonging to 2 classes.


In [40]:
imgs , labels = next(train_batches) # every time it will take another batch of images
labels

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [42]:
batch_size = 128
train_datagen = ImageDataGenerator(rescale=1/255)
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '../YOLLOO/car',  # This is the source directory for training images
        target_size=(200, 200),  # All images will be resized to 200 x 200
        batch_size=batch_size,
        # Specify the classes explicitly
        classes = ['suv','sedan'],
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')


Found 1566 images belonging to 2 classes.


In [41]:
mobile = tf.keras.applications.mobilenet.MobileNet() # import mobilenet

In [42]:
#type(mobile)
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

we are going to take upto total layers - last 6 layers :: That is not including the last 5 layers which we got to know after expermenting in testing no of layers we choose to inlude vs not include
after this no layer will be included : which we do after expermenting "global_average_pooling2d (Gl (None, 1024)" everything else will not be included

10 is for class ::0 to 9 : this is output layer #softmax to give probability distribution
(x)- mobilenet is functional model i.e functional api from keras not sequential api so it calls all previous layer by (x)
basically we r saying pass all the previous layer pass stored in x to output


In [43]:
# Create MOdel
x = mobile.layers[-6].output  #
output = Dense(units=2, activation ="softmax")(x) # prediction= output
model = Model(inputs=mobile.input,outputs=output) # mobile.input taking input from original mobile net model

### This will create new model which is identical to the original model upto 6th last layer as we don't have the last 5 layers
# but instead we have new layer Dense that we added with 2 output nodes which will give prediction

In [44]:
# freeze some layer :  experiment we found that freezing all layers except last 23 layers this will give decent reult
#  Training will be done on last 23 layer only rest is non trainable
# TRAINING 23 LAYER ONLY
for layer in model.layers[:-23]:
    layer.trainable = False

In [45]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

dense_1 (Dense)       (None, 10)                10250    - last layer with 10 class here :: Change it to 2 or 3

In [46]:
#TRAIN THE MODEL
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy', metrics =["accuracy"])

In [47]:
# Train the MODEL
#x : training data, validation data
model.fit(x=train_batches, validation_data = valid_batches, epochs=30,verbose=2)

Epoch 1/30
126/126 - 96s - loss: 0.5998 - accuracy: 0.7220 - val_loss: 1.2814 - val_accuracy: 0.5938
Epoch 2/30
126/126 - 111s - loss: 0.2333 - accuracy: 0.9153 - val_loss: 0.8501 - val_accuracy: 0.7188
Epoch 3/30
126/126 - 107s - loss: 0.1246 - accuracy: 0.9665 - val_loss: 0.2904 - val_accuracy: 0.8750
Epoch 4/30
126/126 - 87s - loss: 0.0940 - accuracy: 0.9728 - val_loss: 0.3842 - val_accuracy: 0.8750
Epoch 5/30
126/126 - 88s - loss: 0.0644 - accuracy: 0.9872 - val_loss: 0.2450 - val_accuracy: 0.9062
Epoch 6/30
126/126 - 85s - loss: 0.0562 - accuracy: 0.9880 - val_loss: 0.3009 - val_accuracy: 0.8750
Epoch 7/30
126/126 - 88s - loss: 0.0629 - accuracy: 0.9880 - val_loss: 0.3232 - val_accuracy: 0.8438
Epoch 8/30
126/126 - 101s - loss: 0.0430 - accuracy: 0.9920 - val_loss: 0.3382 - val_accuracy: 0.9062
Epoch 9/30
126/126 - 106s - loss: 0.0277 - accuracy: 0.9992 - val_loss: 0.3808 - val_accuracy: 0.8750
Epoch 10/30
126/126 - 133s - loss: 0.0300 - accuracy: 0.9920 - val_loss: 0.2398 - val_a

In [55]:
## Run after model traing done
test_labels = test_batches.classes
predictions = model.predict(x=test_batches,verbose =0)
#cm = confusion_matrix(y_true =test_labels, prediction.argmax(axis=1))
test_batches.class_indices

{'suv': 0, 'sedan': 1}

In [53]:
### SAVE THE MODEL####
import os.path

if os.path.isfile('car/SUV_SEDAN_MODEL/suv_sedan_detection.h5') is False:
    model.save('car/SUV_SEDAN_MODEL/suv_sedan_detection.h5')


The save function saves:
-The architecture of the model allowing to recreate the model.
-The weights of the model
-The training configuration(loss, optimizer)
-The state of the optimizer, allowing to resume training where we letf of

In [54]:
#### TO LOAD THE MODEL
from tensorflow.keras.models import load_model
new_model = load_model('car/SUV_SEDAN_MODEL/suv_sedan_detection.h5')
#new_model.summary()
#new_model.get_weights()
#new_model.optimizer

In [57]:
predictions.shape

(282, 2)

In [59]:
len(test_batches)

29

In [60]:
predictions

array([[9.24551487e-01, 7.54484981e-02],
       [1.05574720e-01, 8.94425273e-01],
       [9.99987364e-01, 1.26003233e-05],
       [9.99999881e-01, 1.26593434e-07],
       [1.00000000e+00, 2.29036736e-08],
       [9.99995470e-01, 4.57283795e-06],
       [7.77253509e-01, 2.22746462e-01],
       [9.99631166e-01, 3.68788053e-04],
       [9.99980569e-01, 1.94694530e-05],
       [9.99745071e-01, 2.54882703e-04],
       [6.28236592e-01, 3.71763468e-01],
       [9.01099384e-01, 9.89005938e-02],
       [1.00000000e+00, 3.28491971e-08],
       [9.99995232e-01, 4.72750662e-06],
       [9.99861836e-01, 1.38111121e-04],
       [9.90950942e-01, 9.04900208e-03],
       [9.99880910e-01, 1.19102260e-04],
       [9.99994040e-01, 5.95412257e-06],
       [9.99991298e-01, 8.74419402e-06],
       [9.99899030e-01, 1.00914724e-04],
       [9.95677769e-01, 4.32229741e-03],
       [9.99904513e-01, 9.54634233e-05],
       [7.82916248e-01, 2.17083707e-01],
       [9.99843955e-01, 1.55991860e-04],
       [9.857690